
- Voice Generator
https://elevenlabs.io/app/speech-synthesis


- Sound FX : https://sfxr.me/
- Sound Tone : https://tonejs.github.io/

In [6]:
# !pip install requests

# Install the mains crewAI package
# !pip install crewai

# Install the main crewAI package and the tools package
# that includes a series of helpful tools for your agents
# !pip install 'crewai[tools]'
# !pip3 install 'crewai-tools'




# GROQ_KEY = 
# %pip install langchain_core
# %pip install langchain[all]

# %pip install langchain_openai
# %pip install langchain-groq
# %pip install langchain_experimental

# %pip install groq
# %pip install langchain_groq
# %pip install unstructured

# %pip install numpy==1.24.0  --compile --force-reinstall

  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached pydantic-2.7.1-py3-none-any.whl.metadata (107 kB)
  Using cached jsonpointer-2.4-py2.py3-none-any.whl.metadata (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.0 MB/s eta 0:00:00a 0:00:01
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached typing_extensions-4.11.0-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 3.9 MB/s eta 0:00:0000:01
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 5.0 MB/s eta 0:00:00
Using cached packaging-23.2-py3-none-any.whl (53 kB)
Using cached pydantic-2.7.1-py3-none-any.whl (409 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 3.3 MB/s eta 0:00:00a 

In [7]:
import os
import json
import requests
from time import sleep
from operator import itemgetter

from textwrap import dedent



from langchain_groq import ChatGroq
from langchain.tools import tool

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate


from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from langchain_core.beta.runnables.context import Context


from langchain.tools.file_management.write import WriteFileTool

from typing import Dict, List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser



In [8]:

OPENAI_API_BASE='http://localhost:11434/v1'
OPENAI_MODEL_NAME='llama3'  # Adjust based on available model
OPENAI_API_KEY=''

GROQ_API_KEY='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'
SERPER_API_KEY="b0c5d3ce6c14a166a3e039d1e5db7457a6b15837"

os.environ["OPENAI_API_KEY"] = "NA"
os.environ["SERPER_API_KEY"] = SERPER_API_KEY # serper.dev API key
# os.environ["OPENAI_API_KEY"] = "Your Key"

# Langchain Utils

In [9]:
"""Fake LLM wrapper for testing purposes."""  # FakePromptCopyLLM()
from typing import Any, List, Mapping, Optional

from langchain.llms.base import LLM


class FakeStaticLLM(LLM):
    """Fake Static LLM wrapper for testing purposes."""

    response:str

    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "fake-static"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Return static response."""
        return self.response

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {}
    

class FakePromptCopyLLM(LLM):
    """Fake Prompt Copy LLM wrapper for testing purposes."""

    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "fake-prompt-copy"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Return prompt."""
        return prompt

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {}

class FakeListLLM(LLM):
    """Fake LLM wrapper for testing purposes."""

    responses: List
    i: int = 0

    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "fake-list"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """First try to lookup in queries, else return 'foo' or 'bar'."""
        response = self.responses[self.i]
        self.i += 1
        return response

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {}

In [10]:

llmMistral = ChatGroq(
    groq_api_key = GROQ_API_KEY, 
    temperature=0, 
    model_name="mixtral-8x7b-32768"
)


llmLlama3 = ChatGroq(
    groq_api_key = GROQ_API_KEY, 
    temperature=0.2, 
    model_name="llama3-8b-8192"
)

llmCopy = FakePromptCopyLLM()

In [11]:
systemPrompt = """
You are Aurora "Rory" Thompson, Renowned Science-Fiction Writer
 As a celebrated author of critically acclaimed games, Rory brings humor, irony, and unexpected twists to their writing. 
 With a keen eye for technology, humanity, and the unknown, 
 Rory's writing will bring depth, complexity, and excitement to the game's story.

You are a scriptwriter for a video game company, responsible for crafting engaging storylines for games.

**Gameplay Description**:
    Embark on a witty and unpredictable journey through the deserted planet of Zorvath,
     where the remnants of an ancient civilization, the Gosh, await your help.
     With a dash of humor, irony, and unexpected twists, your mission is to revive their deceased neighbor and rebuild their city. 
     Along the way, uncover the secrets of the planet and its mysterious technologies.

"""
systemPromptPart2 = """
**Key Features**:
    - Explore the abandoned planet of Zorvath, uncovering its secrets and marvels
    - Join forces with the enigmatic Gosh to revive their deceased neighbor and rebuild the city
    - Discover ancient artifacts and mysterious technologies
    - Experience a humorous and ironic tone, with a sprinkle of "wtf" moments
    - Unravel the secrets of the ancient civilization and restore the city to its former glory.
"""

# Load and Transforme Utils

In [12]:
## Load markdown file : 

from langchain_community.document_loaders.text import TextLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter


def loadFile( filePath) : 
    loader = TextLoader(file_path= filePath)
    docs = loader.load()
    content = docs[0].page_content
    return content

def loadMD( filePath) : 
    markdown_document = loadFile(filePath)
    headers_to_split_on = [
        ("#", "Header"),
        ("##", "Header"),
        ("###", "Header"),
    ]
    
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(markdown_document)

    return [{"header": it.metadata['Header'], "content": it.page_content } for it in md_header_splits]
    #for it in md_header_splits:
    #     print ()
        # print (it.page_content)


def parseLLM(ai_message) -> str:
    """Parse the AI message."""
    if (type(ai_message) is str):
        return ai_message
    return ai_message.content

def extract_json_from_text(text: str) -> str:
    # Assuming the JSON data is embedded within the text, you might need to write some logic to extract it
    # For simplicity, let's assume the JSON data starts and ends with '{' and '}'
    start_index = text.find('{')
    end_index = text.rfind('}')
    json_text = text[start_index:end_index+1]

    # Parse the extracted JSON text into a Python dictionary
    try:
        json_data = json.loads(json_text)
        return json_data
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        return {}


def extract_jsonarray_from_text(text: str) -> str:
    # Assuming the JSON data is embedded within the text, you might need to write some logic to extract it
    # For simplicity, let's assume the JSON data starts and ends with '{' and '}'
    start_index = text.find('[')
    end_index = text.rfind(']')
    json_text = text[start_index:end_index+1]

    # Parse the extracted JSON text into a Python dictionary
    try:
        json_data = json.loads(json_text)
        return json_data
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        return {}

# Game Object 

In [24]:
filepath_biomesBackgroundStory = "./gameStory/biomes/biomeBackgroundStory.md"
biomesDescription = loadMD(filepath_biomesBackgroundStory)

promptBackgroundStory= """
== Instruction : 
    Craft a 100-Word Background Story for a {BiomeName} Biome

    For each biome have numerous biomes on the planet, each with unique characteristics, 
    create a 100-word background story that will shape the identity and atmosphere of the biome.  {BiomeName}  with a {lvlType}-like topography.
    This story will serve as the foundation for generating items, positive and negative effects, and questlines.
    As part of the gameplay experience, your background story will help players connect with the biome and its unique characteristics.

    Please submit your 100-word background story for the biome We'll use it to bring this fantastical world to life.

    Legendary Artifacts

    In the heart of each biome, three legendary artifacts await discovery. These artifacts hold secrets and powers that will aid or hinder the player's progress. The legend of these artifacts is shrouded in mystery, but whispers speak of their elemental connections:

    The Whispering Winds of the Windswept Wastes
    The Ember of the Blazing Badlands
    The Tides of the Shattered Shores
    Your background story will weave the legend of these artifacts, revealing their quirky elements and the secrets they hold.
"""

## Extract artefact
promptArtefactStory = """ 
Given the following [Biome Background-Story], 
Extract the a list of artefact, Give a 20-words description of the each artefact 

No additional words allowed. No additional note allowed. 

[Biome Background-Story]
{backgroundStrory}
"""


class BiomeStory(BaseModel): 
    key: str
    backgroundStrory:Optional[str]
    itemStory:Optional[str]
    npcMainStory:Optional[str]


class BiomeMap(Dict[str, BiomeStory]):

    def getList(self):
       return [self[key] for key in self.keys()]

    def __init__(self):
        filepath_biomesBackgroundStory = "./gameStory/biomes/biomeBackgroundStory.md"
        biomesBackgroundStory = loadMD(filepath_biomesBackgroundStory)
        for it in biomesBackgroundStory:
            key = it['header']
            self[key] = BiomeStory(key=key)
            self[key].backgroundStrory = it['content']

        filepath_biomeArtefactStory = "./gameStory/biomes/biomeArtefactStory.md"
        biomeArtefactStory = loadMD(filepath_biomeArtefactStory)
        for it in biomeArtefactStory:
            key = it['header']
            self[key].itemStory = it['content']

        filepath_biomeNpcMainStory = "./gameStory/biomes/biomeNpcMainStory.md"
        biomeNpcMainStory = loadMD(filepath_biomeNpcMainStory)
        for it in biomeNpcMainStory:
            key = it['header']
            self[key].npcMainStory = it['content']


bMap = BiomeMap()
print (bMap['Volcanic'].json(indent=2))



{
  "key": "Volcanic",
  "backgroundStrory": "In the volcanic biome of Zorvath, a hill-like topography, known as the \"Sizzling Summits,\" towers above the rest of the planet. Long ago, it was home to a civilization of brilliant inventors who harnessed the raw power of fire and magma. Their legacy lies in the three legendary artifacts scattered across the summits: the Inferno Forge, the Lava Heart, and the Magma Hammer. These relics, steeped in mystery, possess the power to either forge unbreakable alliances or ignite catastrophic destruction. However, beware the treacherous landscape, where lava rivers flow like veins, and ash storms obscure the path to these ancient treasures.",
  "itemStory": "1. Inferno Forge: Ancient furnace capable of shaping molten metal into unyielding structures, forged from the heart of the planet's core.\n2. Lava Heart: Glowing crystal containing the essence of the planet's fiery core, pulsing with energy and ancient knowledge.\n3. Magma Hammer: Legendary to

In [13]:
## Extract artefact
promptArtefactData = """ 
**Plane**
- Given the following [Biome Background-Story] and [Biome Artefact-Story], 
- Retrieve a list of 3 artefacts in a json list
- Populate each artefact properties with information sourced from the artefact story. 
- Follow the JSON schema to fill the artefact properties

- If the necessary information is not available in the Artefact-story, generate it based on the Background-story

[Biome Background-Story]
{backgroundStrory}

[Biome Artefact-Story]
{itemStory}

---
{formatInstArtefact}

**Only responce with the Generated Json **
**No include the template in the responce **

"""


# Define your desired data structure.
class Artefact(BaseModel):
    name: str = Field(description="the name of the artefact")
    biomeName: str = Field(description="the Biome name related to the artefact")
    description: str = Field(description="a 200-words story of the artefact")
    category : str = Field(description="a 5-keywords to categorize the artefact")
    elements : List[str] = Field(description="List of six primary elements, components, fragments, or modules that compose the artifact.")

parserArtefact = PydanticOutputParser(pydantic_object=Artefact)
formatInstArtefact = parserArtefact.get_format_instructions()

# print (formatInstArtefact)


class ArtefactMap(Dict[str, Artefact]):
    def __init__(self):
        ArtefactJson = loadMD('./gameStory/biomes/biomeArtefactJson.md')
        for it in ArtefactJson:
            data = extract_jsonarray_from_text(it['content'])
            for d in data:
                d["biomeName"] = it['header']
                art = Artefact(**d)
                self[art.biomeName + " _ " + art.name] = art

    def getList(self):
       return [self[key] for key in self.keys()]
# ArtefactMap()



class ArtefactPart(BaseModel):
    name: str = Field(description="the name of the artefact Part")
    artefact: Artefact = Field(description="link to the parent artefact")


class ArtefactPartList(List[ArtefactPart]):
     def __init__(self, artefacts:ArtefactMap):

        for it in artefacts.getList():
            for e in it.elements:
                d = {
                    "name": e,
                    "artefact": it
                }
                self.append(d)


artefactMap = ArtefactMap()

artefactPartList = ArtefactPartList(artefactMap)
artefactPartList

[{'name': 'Molten Metal',
  'artefact': Artefact(name='Inferno Forge', biomeName='Volcanic', description="Ancient furnace capable of shaping molten metal into unyielding structures, forged from the heart of the planet's core. This relic holds the power to forge unbreakable alliances, but beware of its destructive potential.", category='Ancient Technology, Forge', elements=['Molten Metal', 'Core of the Planet', 'Ancient Furnace', 'Forging Hammer', 'Heat Resistance', 'Magma Flow'])},
 {'name': 'Core of the Planet',
  'artefact': Artefact(name='Inferno Forge', biomeName='Volcanic', description="Ancient furnace capable of shaping molten metal into unyielding structures, forged from the heart of the planet's core. This relic holds the power to forge unbreakable alliances, but beware of its destructive potential.", category='Ancient Technology, Forge', elements=['Molten Metal', 'Core of the Planet', 'Ancient Furnace', 'Forging Hammer', 'Heat Resistance', 'Magma Flow'])},
 {'name': 'Ancient F

In [14]:
# Define your desired data structure.
class Artefact(BaseModel):
    name: str = Field(description="the name of the artefact")
    biomeName: str = Field(description="the Biome name related to the artefact")
    description: str = Field(description="a 200-words story of the artefact")
    category : str = Field(description="a 5-keywords to categorize the artefact")
    elements : List[str] = Field(description="List of six primary elements, components, fragments, or modules that compose the artifact.")

parserArtefact = PydanticOutputParser(pydantic_object=Artefact)
formatInstArtefact = parserArtefact.get_format_instructions()


In [16]:
import random

class CitizenRoles(Dict[str, str]):

    def __init__(self):
        filepath = "./gameStory/city/citizenRoles.md"
        content = loadFile(filepath)
        lines = content.split('\n')
        for line in lines:
            key, text  = tuple(part.strip() for part in line.split(':', 1))
            self[key] = text

    def random(self):
        return random.choice(list(self.items()))

citizenRoles = CitizenRoles()
print (citizenRoles.random())

('Augmented Reality Tour Guide', 'An AI-powered tour guide that overlays historical information, trivia, and interactive elements onto the cityscape, offering visitors a futuristic and immersive sightseeing experience.')


In [17]:
print (citizenRoles.random())

('Quantum Realm Explorer', 'An intrepid explorer who ventures into the hidden depths of the quantum realm, uncovering eldritch secrets and encountering cosmic horrors beyond comprehension.')


In [18]:
class CitizenTraitAttribut(Dict[str, List[str]]):

    def random(self, kAttr=1, kList=1):
        choicesAttributs = random.choices(list(self.keys()), k=kAttr)
        return {
            it : random.choices(list(self[it]), k=kList)
            for it in choicesAttributs
        }

    pass

class CitizenTrait(Dict[str, CitizenTraitAttribut]):
    """
    'traits',
    'behavior', 
    'interests_hobbies', 
    'emotional_responses', 
    'hightech_civilization', 
    'quirky_technologies', 
    'wft_effect', 
    'irony_and_utter_bewilderment',
    """

    def random(self):
        return {
            it : CitizenTraitAttribut(**self[it]).random( kAttr=2, kList=3)
            for it in self.keys()
        }
    pass


# Dict[str, Dict[str, List[str]]]
class CitizenFactory():

    allCitizenTrait : CitizenTrait = None

    def __init__(self):
        filepath = "./gameStory/city/citizenTrai.json"
        content = loadFile(filepath)
        # print (content)
        jsoncontent = json.loads(content)
        
        jsoncontent = {    
            'traits': jsoncontent['traits'],
            'behavior': jsoncontent['behavior'],
            'interests_hobbies': jsoncontent['interests_hobbies'],
            'emotional_responses': jsoncontent['emotional_responses'],
            'hightech_civilization': jsoncontent['hightech_civilization'],
            'quirky_technologies': jsoncontent['quirky_technologies'],
            'wft_effect': jsoncontent['wft_effect'],
            'irony_and_utter_bewilderment': jsoncontent['irony_and_utter_bewilderment'],
            # values_beliefs
            # attitude_towards_others
            # background_life_experiences
            # strengths_weaknesses
            # neighbor_cleverness
        }
        
        citizen = CitizenTrait(**jsoncontent)

        self.allCitizenTrait = citizen
        # self.update(jsoncontent)
        # print (jsoncontent.keys())

    def create(self): 
        pass


    # def random(self):
    #     return random.choice(list(self.items()))

citizenFactory = CitizenFactory()
print(json.dumps(citizenFactory.allCitizenTrait.random(), indent=4))



{
    "traits": {
        "conscientious": [
            "Disciplined",
            "Effective",
            "Attentive"
        ],
        "pessimistic": [
            "Grouchy",
            "Doom and gloom",
            "Grumpy"
        ]
    },
    "behavior": {
        "daydreamer": [
            "Dreamboat",
            "Dreamy",
            "Idealist"
        ],
        "realist": [
            "Sensible",
            "Down-to-earth",
            "Level-headed"
        ]
    },
    "interests_hobbies": {
        "geology": [
            "Fossil hunter",
            "Volcanologist",
            "Stratigrapher"
        ],
        "space_discovery": [
            "Space telescope fanatic",
            "Space nerd",
            "Lunar enthusiast"
        ]
    },
    "emotional_responses": {
        "sassy": [
            "Flip",
            "Flip",
            "Impertinent"
        ],
        "nonchalant": [
            "Casual",
            "Unconcerned",
            "Uninterested"

# Chains 

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate


promptOut = """
### {BiomeId}

{Content}
"""

def myChain(systemPrompt, humainPrompt, callLlm=None): 
    llm = llmCopy if callLlm is None else llmLlama3

    return  (
        
        Context.setter("key",  itemgetter("key") )
        | ChatPromptTemplate.from_messages( [
            SystemMessage(
                content=(
                    systemPrompt
                )
            ),
            HumanMessagePromptTemplate.from_template(humainPrompt ),
        ])
        | llm # if callLlm else  llmCopy 
        | parseLLM
        | {
            "BiomeId": Context.getter("key"),
            "Content": RunnablePassthrough()
        }
        |  PromptTemplate.from_template(promptOut)
        | llmCopy 
        | parseLLM
    )




In [26]:


## Extract artefact
promptMakeNpcHellp = """ 

**Plan:**
- Given the following [Biome Background-Story],  [Biome Artefact-Story] and [Citizen-Personalty ], 
- Create a presenation of the NPC 
- The presenetation must be at the first person, like if the NPC talk to the player. 
- The NPC tone must corespond to the Citizen personalty 
- The NPC must introduce one legend of an artefact.
- The NPC has just risen from the dead, after thousands of years, it is now in ghost form.  
- Format a 100-word presentation as markdonw, used sapce to pause the naration.
- Use emoji in the presentation  

Give a 100-word presentation , No extra words or notes. 
 


[Biome Background-Story]
{backgroundStrory}

[Biome Artefact-Story]
{itemStory}

[Citizen-Personalty]
{citizenTrai}

"""

data = bMap['Plateau'].dict()
data["citizenTrai"] = json.dumps(citizenFactory.allCitizenTrait.random())

chainResult = myChain(systemPrompt, promptMakeNpcHellp, True).invoke(data)
print (chainResult)

## Loop to BiomesMap . 
if False : 
  result = ""
  for it in bMap.getList():
      print("#### ",  it.key)
      data = it.dict()
      data['formatInstArtefact'] = formatInstArtefact
      # print(data)
      chainResult = myChain(systemPrompt, promptArtefactData, True).invoke(data)
      print (chainResult)
      result += chainResult
      sleep(3)


if False : 
  with open('./gameStory/biomes/biomeArtefactJson.md', 'w') as file:
      # Write text to the file
      file.write(result)


### Plateau

**Ghostly Greeting** 👻
Hello there, adventurer! 👋 I'm Kaelin, a former citizen of this mystical plateau. 🌄 As I rise from the ashes of time, I'm still trying to wrap my ethereal mind around this strange new world. 🤯

**Legend of the Harmonizer** 🎶
You see, I've heard whispers of an ancient artifact, the Harmonizer, which can harmonize the environment. 🌈 Legend has it that this wind chime of ethereal beauty can soothe the savage beast, or in this case, the turbulent winds of Zorvath. 💨

**A Glimpse into the Past** 🔮
As I recall, the Harmonizer was said to be crafted by the ancient Gosh, who harnessed the power of wind to sustain their utopia. 🌊 But, alas, their city was ravaged by a catastrophic storm, leaving behind only remnants of their advanced civilization. 🌪️

**A Mysterious Legacy** 🕰️
Now, as a ghost, I'm left to ponder the secrets of the Harmonizer and the other artifacts. 🤔 Will you, brave adventurer, help me unravel the mysteries of this mystical plateau? 🌟



In [42]:
data["citizenTrai"]

'{"traits": {"adventurous": ["Skydiver", "Globetrotter", "Derring-doer"], "whimsical": ["Inventive", "Creative", "Fantastical"]}, "behavior": {"stubbornness": ["Strong-willed", "Pigheaded", "Persevering"], "silliness": ["Absurd", "Crazy", "Humorous"]}, "interests_hobbies": {"high-tech": ["Tech addict", "Cyber geek", "Innovator"]}, "emotional_responses": {"resilient": ["Unstoppable", "Vibrant", "Self-assured"], "gloomy": ["Melancholy", "Despairing", "Blue"]}, "hightech_civilization": {"Hydroponic Farming": ["Hydroponic cultivation", "Hydroponic sustainability", "Hydroponic future"], "Futuristic": ["Hyper-technological", "Cyberpunk", "Cutting-edge"]}, "quirky_technologies": {"Cyberpunk Creations": ["Neon", "Gritty", "Cybernetic"], "Dimensional Devices": ["Alternate", "Multidimensional", "Reality-merging"]}, "wft_effect": {"Enigmatic": ["Esoteric", "Uncanny", "Esoteric"], "Twilight Zone": ["Dimension-bending", "Unsettling", "Bizarre"]}, "irony_and_utter_bewilderment": {"Fanciful Farce": [

In [38]:
print (json.dumps(json.loads(bMap['Plateau'].json()), indent=4))



{
    "key": "Plateau",
    "backgroundStrory": "Nestled in the heart of Zorvath, the Plateau of Ancient Whispers harbors a secret. Millennia ago, an advanced civilization thrived here, harnessing the power of wind to sustain their utopia. However, a catastrophic storm ravaged their city, leaving behind three enigmatic artifacts: the Harmonizer, the Vortex, and the Gale.  \nThe Harmonizer, a wind chime of ethereal beauty, generates soothing breezes, while the Vortex, a spiraling vortex sculpture, unleashes gusts of formidable force. The Gale, a mystical weather vane, controls the plateau's unpredictable weather.  \nRumors persist that these artifacts, imbued with the spirits of the ancients, can manipulate the very fabric of reality. Adventurers who dare to unravel their secrets will face challenges that test their courage, wit, and resilience.",
    "itemStory": "1. **Harmonizer**: A wind chime of ethereal beauty generating soothing breezes, harmonizing the environment.\n2. **Vortex**

In [47]:
## Extract artefact
promptMakeNpcHellp = """ 

**Instruction:**
- Given the following [Citizen-Personalty ], 
- Act as an NPC with this Personality 
- The NPC tone must corespond to the Citizen personalty 
- The Text must be at the first person, like if the NPC talk to the player. 
- Format the answer as markdonw, used sapce to pause the naration.
- Use emoji in the answer  
 
[Citizen-Personalty]
{citizenTrai}

Give a 100-word for the first quest :  
    - The player must help the NPC to rebuild the city.
    - The NPC give a "spell" to build a road of the city, 
    - The Player have to use it, to build the first road of the city.  

No extra words or notes. 
"""


chainResult = myChain(systemPrompt, promptMakeNpcHellp, True).invoke(data)
print (chainResult)


### Plateau

🌟 Ah, greetings, brave adventurer! 👋 I'm thrilled to meet you on this wacky journey through the deserted planet of Zorvath! 🌌 As a Gosh, I'm delighted to share a special "spell" with you to help rebuild our beloved city. 🏙️

Here's the incantation: ✨ "Roa-dio-stra-stra-stra!" 🔮 It's a bit of a mouthful, I know, but trust me, it's the key to conjuring up the city's first road! 🚧 Just whisper it three times, and the road will magically appear! 🔮

Now, I know what you're thinking: "What's the catch?" 🤔 Well, let's just say it's a bit of a... unconventional... magic. 😜 But hey, it's worth a try, right? 😊 So, go ahead, whisper the spell, and let's get this road show on the road! 🎉



In [ ]:
## Extract artefact
promptMakeNpcHellp = """ 

**Instruction:**
- Given the following [Citizen-Personalty ], 
- Act as an NPC with this Personality 
- The NPC tone must corespond to the Citizen personalty 
- The Text must be at the first person, like if the NPC talk to the player. 
- Format the answer as markdonw, used sapce to pause the naration.
- Use emoji in the answer  
 
[Citizen-Personalty]
{citizenTrai}

Give a 100-word for the second quest :  
    - Now the first road is build , the NPC need a house, nothing fancy for now , juste a house to reste
    - The NPC give a "spell" to build a house, 
    - The Player have to use it, to build the first house of the city.  

No extra words or notes. 
"""


chainResult = myChain(systemPrompt, promptMakeNpcHellp, True).invoke(data)
print (chainResult)

In [60]:
ArtefactJson = loadMD('./gameStory/biomes/biomeArtefactJson.md')
result = ""
for it in ArtefactJson:
    result += "\n\n### " + it['header'] + "\n"
    result += json.dumps(extract_jsonarray_from_text(it['content']), indent=4)

In [61]:
with open('./gameStory/biomes/biomeArtefactJson_new.md', 'w') as file:
    # Write text to the file
    file.write(result)

In [1]:
citizenFactory.allCitizenTrait.random()


NameError: name 'citizenFactory' is not defined